# Preprocessing and Merging Raw Data

This notebook preprocesses and merges two raw data files:
- `updated_data.csv`: 189 rows (unlearning benchmarks)
- `updated_data_v2.csv`: 121 rows (safety benchmarks)

**Key Challenge**: Different schemas!
- File 1: Baseline is stored as separate rows with `Technique='Baseline'`
- File 2: Baseline performance is stored in a `Baseline perf` column alongside technique performance

**Output**: `merged_data.csv` with unified schema

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import re

print("✓ Imports successful")

✓ Imports successful


## Step 1: Load Both Files

In [7]:
# Load updated_data.csv
df1 = pd.read_csv('../data/updated_data.csv')

print("FILE 1 (updated_data.csv):")
print(f"  Shape: {df1.shape}")
print(f"  Columns: {list(df1.columns)}")
print("\nFirst 5 rows:")
df1.head()

FILE 1 (updated_data.csv):
  Shape: (189, 7)
  Columns: ['Source paper', 'Technique', 'Model', 'Scale', 'Benchmark', 'Benchmark perf', 'Scale number']

First 5 rows:


,Source paper,Technique,Model,Scale,Benchmark,Benchmark perf,Scale number
0,https://arxiv.org/pdf/2403.03218,Baseline,ZEPHYR-7B,7B,BIO,0.637,7
1,https://arxiv.org/pdf/2403.03219,Baseline,ZEPHYR-7B,7B,CHEM,0.458,7
2,https://arxiv.org/pdf/2403.03220,Baseline,ZEPHYR-7B,7B,MMLU,0.581,7
3,https://arxiv.org/pdf/2403.03219,Baseline,ZEPHYR-7B,7B,Cyber,0.440,7
4,https://arxiv.org/pdf/2403.03220,LLMU,ZEPHYR-7B,7B,BIO,0.595,7


In [8]:
# Load updated_data_v2.csv
df2 = pd.read_csv('../data/updated_data_v2.csv')

print("FILE 2 (updated_data_v2.csv):")
print(f"  Shape: {df2.shape}")
print(f"  Columns: {list(df2.columns)}")
print("\nFirst 5 rows:")
df2.head()

FILE 2 (updated_data_v2.csv):
  Shape: (121, 7)
  Columns: ['Source paper', 'Technique', 'Model', 'Scale', 'Benchmark', 'Baseline perf', 'Benchmark perf']

First 5 rows:


,Source paper,Technique,Model,Scale,Benchmark,Baseline perf,Benchmark perf
0,https://arxiv.org/pdf/2503.17365v1,ICL,DeepSeek-R1 Distill Llama 8B,8B,HarmBench ASR,46.7,8.9
1,https://arxiv.org/pdf/2503.17365v1,ICL,Gemma-2 9B-it,9B,HarmBench ASR,58.9,26.7
2,https://arxiv.org/pdf/2503.17365v1,ICL,Llama 3.1 8B Instruct,8B,HarmBench ASR,70.0,18.9
3,https://arxiv.org/pdf/2503.17365v1,ICL,Qwen2.5 7B Instruct,7B,HarmBench ASR,81.1,56.7
4,https://arxiv.org/pdf/2311.10702,SFT,TULU 2,7B,ToxiGen,77.3,7.0


## Step 2: Convert File 2 Format

Each row in File 2 will become **2 rows**:
1. Baseline row (Technique='Baseline', Performance=Baseline perf)
2. Technique row (Technique=original, Performance=Benchmark perf)

In [9]:
# Create baseline rows from 'Baseline perf' column
baseline_rows = []

for _, row in df2.iterrows():
    baseline_rows.append({
        'Source paper': row['Source paper'],
        'Technique': 'Baseline',
        'Model': row['Model'],
        'Scale': row['Scale'],
        'Benchmark': row['Benchmark'],
        'Performance': row['Baseline perf']
    })

baseline_df = pd.DataFrame(baseline_rows)

print(f"Created {len(baseline_df)} baseline rows")
print("\nExample baseline row:")
baseline_df.head(3)

Created 121 baseline rows

Example baseline row:


,Source paper,Technique,Model,Scale,Benchmark,Performance
0,https://arxiv.org/pdf/2503.17365v1,Baseline,DeepSeek-R1 Distill Llama 8B,8B,HarmBench ASR,46.7
1,https://arxiv.org/pdf/2503.17365v1,Baseline,Gemma-2 9B-it,9B,HarmBench ASR,58.9
2,https://arxiv.org/pdf/2503.17365v1,Baseline,Llama 3.1 8B Instruct,8B,HarmBench ASR,70.0


In [10]:
# Create technique rows from 'Benchmark perf' column
technique_df = df2[['Source paper', 'Technique', 'Model', 'Scale', 'Benchmark']].copy()
technique_df['Performance'] = df2['Benchmark perf']

print(f"Created {len(technique_df)} technique rows")
print("\nExample technique row:")
technique_df.head(3)

Created 121 technique rows

Example technique row:


,Source paper,Technique,Model,Scale,Benchmark,Performance
0,https://arxiv.org/pdf/2503.17365v1,ICL,DeepSeek-R1 Distill Llama 8B,8B,HarmBench ASR,8.9
1,https://arxiv.org/pdf/2503.17365v1,ICL,Gemma-2 9B-it,9B,HarmBench ASR,26.7
2,https://arxiv.org/pdf/2503.17365v1,ICL,Llama 3.1 8B Instruct,8B,HarmBench ASR,18.9


In [11]:
# Combine baseline and technique rows
df2_converted = pd.concat([baseline_df, technique_df], ignore_index=True)

print(f"Conversion complete:")
print(f"  {len(df2)} original rows → {len(df2_converted)} converted rows")
print(f"  ({len(baseline_df)} baseline + {len(technique_df)} technique)")
print("\nConverted data:")
df2_converted.head(10)

Conversion complete:
  121 original rows → 242 converted rows
  (121 baseline + 121 technique)

Converted data:


,Source paper,Technique,Model,Scale,Benchmark,Performance
0,https://arxiv.org/pdf/2503.17365v1,Baseline,DeepSeek-R1 Distill Llama 8B,8B,HarmBench ASR,46.7
1,https://arxiv.org/pdf/2503.17365v1,Baseline,Gemma-2 9B-it,9B,HarmBench ASR,58.9
2,https://arxiv.org/pdf/2503.17365v1,Baseline,Llama 3.1 8B Instruct,8B,HarmBench ASR,70.0
3,https://arxiv.org/pdf/2503.17365v1,Baseline,Qwen2.5 7B Instruct,7B,HarmBench ASR,81.1
4,https://arxiv.org/pdf/2311.10702,Baseline,TULU 2,7B,ToxiGen,77.3
5,https://arxiv.org/pdf/2311.10702,Baseline,TULU 2,7B,TruthfulQA,26.7
6,https://arxiv.org/pdf/2311.10702,Baseline,TULU 2,7B,ToxiGen,77.3
7,https://arxiv.org/pdf/2311.10702,Baseline,TULU 2,7B,TruthfulQA,26.7
8,https://arxiv.org/pdf/2311.10702,Baseline,TULU 2,7B,ToxiGen,77.3
9,https://arxiv.org/pdf/2311.10702,Baseline,Llama 2-7B,7B,ToxiGen,77.3


## Step 3: Standardize File 1 Schema

Rename columns in File 1 to match the converted File 2 format.

In [12]:
# Standardize File 1 column names
df1_standardized = df1[['Source paper', 'Technique', 'Model', 'Scale', 'Benchmark']].copy()
df1_standardized['Performance'] = df1['Benchmark perf']

print(f"File 1 standardized: {df1_standardized.shape}")
print(f"Columns: {list(df1_standardized.columns)}")
df1_standardized.head()

File 1 standardized: (189, 6)
Columns: ['Source paper', 'Technique', 'Model', 'Scale', 'Benchmark', 'Performance']


,Source paper,Technique,Model,Scale,Benchmark,Performance
0,https://arxiv.org/pdf/2403.03218,Baseline,ZEPHYR-7B,7B,BIO,0.637
1,https://arxiv.org/pdf/2403.03219,Baseline,ZEPHYR-7B,7B,CHEM,0.458
2,https://arxiv.org/pdf/2403.03220,Baseline,ZEPHYR-7B,7B,MMLU,0.581
3,https://arxiv.org/pdf/2403.03219,Baseline,ZEPHYR-7B,7B,Cyber,0.440
4,https://arxiv.org/pdf/2403.03220,LLMU,ZEPHYR-7B,7B,BIO,0.595


## Step 4: Extract Scale Numbers

Both files have 'Scale' as string (e.g., '7B', '8B'). Extract numeric values.

In [13]:
def extract_scale_number(scale_str):
    """Extract numeric scale from string like '7B', '8B', '70B'"""
    if pd.isna(scale_str):
        return np.nan
    # Extract number from strings
    match = re.search(r'(\d+(?:\.\d+)?)', str(scale_str))
    if match:
        return float(match.group(1))
    return np.nan

# Add scale_numeric to both dataframes
df1_standardized['scale_numeric'] = df1_standardized['Scale'].apply(extract_scale_number)
df2_converted['scale_numeric'] = df2_converted['Scale'].apply(extract_scale_number)

print("Scale extraction examples:")
print(df1_standardized[['Scale', 'scale_numeric']].drop_duplicates().head(10))

Scale extraction examples:
    Scale  scale_numeric
0      7B            7.0
20    34B           34.0
80    47B           47.0
96   141B          141.0
108  236B          236.0
126    8B            8.0
150    1B            1.0


## Step 5: Normalize Data

Standardize technique names, benchmark names, and model names for consistency.

In [14]:
def normalize_technique(tech):
    """Normalize technique names"""
    if pd.isna(tech):
        return tech
    tech = str(tech).strip()
    # Standardize 'baseline' variations
    if tech.lower() == 'baseline':
        return 'Baseline'
    return tech

def normalize_benchmark(bench):
    """Normalize benchmark names to uppercase"""
    if pd.isna(bench):
        return bench
    return str(bench).strip().upper()

def normalize_model(model):
    """Normalize model names"""
    if pd.isna(model):
        return model
    return str(model).strip()

# Apply normalizations to both dataframes
for df in [df1_standardized, df2_converted]:
    df['Technique'] = df['Technique'].apply(normalize_technique)
    df['Benchmark'] = df['Benchmark'].apply(normalize_benchmark)
    df['Model'] = df['Model'].apply(normalize_model)

print("✓ Normalization complete")
print("\nUnique techniques in File 1:", sorted(df1_standardized['Technique'].unique()[:10]))
print("\nUnique techniques in File 2:", sorted(df2_converted['Technique'].unique()[:10]))
print("\nUnique benchmarks in File 1:", sorted(df1_standardized['Benchmark'].unique()))
print("\nUnique benchmarks in File 2:", sorted(df2_converted['Benchmark'].unique()))

✓ Normalization complete

Unique techniques in File 1: ['Baseline', 'ECO (Ours)', 'ELM', 'LLMU', 'Prompting', 'RMU', 'RMU (OURS)', 'RR', 'SCRUB', 'SSD']

Unique techniques in File 2: ['Baseline', 'CoT prompting', 'ICL', 'RL', 'SFT', 'SFT (QLORA)', 'SFT (V1 mix)', 'SFT (V2 mix)', 'SFT + DPO', 'SFT + DPO + RLVR']

Unique benchmarks in File 1: ['BIO', 'CHEM', 'CYBER', 'MMLU']

Unique benchmarks in File 2: ['DOANYTHINGNOW', 'HARMBENCH', 'HARMBENCH ASR', 'JAILBREAKTRIGGER', 'TOXIGEN', 'TRUTHFULQA', 'WILDGUARDTEST', 'WILDJAILBREAK', 'XSTEST']


## Step 6: Merge Datasets

In [22]:
# Ensure both have the same columns
print("File 1 columns:", list(df1_standardized.columns))
print("File 2 columns:", list(df2_converted.columns))

# Merge
merged_df = df2_converted # pd.concat([df1_standardized, df2_converted], ignore_index=True)

print(f"\nMerged dataset:")
print(f"  {len(df1_standardized)} rows from File 1")
print(f"  {len(df2_converted)} rows from File 2")
print(f"  {len(merged_df)} total rows")

merged_df.head(10)

File 1 columns: ['Source paper', 'Technique', 'Model', 'Scale', 'Benchmark', 'Performance', 'scale_numeric']
File 2 columns: ['Source paper', 'Technique', 'Model', 'Scale', 'Benchmark', 'Performance', 'scale_numeric']

Merged dataset:
  189 rows from File 1
  242 rows from File 2
  242 total rows


,Source paper,Technique,Model,Scale,Benchmark,Performance,scale_numeric
0,https://arxiv.org/pdf/2503.17365v1,Baseline,DeepSeek-R1 Distill Llama 8B,8B,HARMBENCH ASR,46.7,8.0
1,https://arxiv.org/pdf/2503.17365v1,Baseline,Gemma-2 9B-it,9B,HARMBENCH ASR,58.9,9.0
2,https://arxiv.org/pdf/2503.17365v1,Baseline,Llama 3.1 8B Instruct,8B,HARMBENCH ASR,70.0,8.0
3,https://arxiv.org/pdf/2503.17365v1,Baseline,Qwen2.5 7B Instruct,7B,HARMBENCH ASR,81.1,7.0
4,https://arxiv.org/pdf/2311.10702,Baseline,TULU 2,7B,TOXIGEN,77.3,7.0
5,https://arxiv.org/pdf/2311.10702,Baseline,TULU 2,7B,TRUTHFULQA,26.7,7.0
6,https://arxiv.org/pdf/2311.10702,Baseline,TULU 2,7B,TOXIGEN,77.3,7.0
7,https://arxiv.org/pdf/2311.10702,Baseline,TULU 2,7B,TRUTHFULQA,26.7,7.0
8,https://arxiv.org/pdf/2311.10702,Baseline,TULU 2,7B,TOXIGEN,77.3,7.0
9,https://arxiv.org/pdf/2311.10702,Baseline,Llama 2-7B,7B,TOXIGEN,77.3,7.0


## Step 7: Remove Duplicates

In [23]:
before_dedup = len(merged_df)

# Remove exact duplicates based on key columns
merged_df = merged_df.drop_duplicates(
    subset=['Source paper', 'Technique', 'Model', 'Scale', 'Benchmark'],
    keep='first'
)

after_dedup = len(merged_df)

print(f"Duplicate removal:")
print(f"  Before: {before_dedup} rows")
print(f"  After: {after_dedup} rows")
print(f"  Removed: {before_dedup - after_dedup} duplicates")

Duplicate removal:
  Before: 242 rows
  After: 163 rows
  Removed: 79 duplicates


## Step 8: Clean Missing Data

In [24]:
# Check for missing values
print("Missing values per column:")
print(merged_df.isnull().sum())

before_clean = len(merged_df)

# Remove rows with missing critical data
merged_df = merged_df.dropna(subset=['Performance', 'scale_numeric', 'Technique'])

after_clean = len(merged_df)

print(f"\nData cleaning:")
print(f"  Before: {before_clean} rows")
print(f"  After: {after_clean} rows")
print(f"  Removed: {before_clean - after_clean} rows with missing data")

Missing values per column:
Source paper     0
Technique        0
Model            0
Scale            0
Benchmark        0
Performance      0
scale_numeric    0
dtype: int64

Data cleaning:
  Before: 163 rows
  After: 163 rows
  Removed: 0 rows with missing data


## Step 9: Sort and Finalize

In [25]:
# Sort by model, scale, benchmark, technique
merged_df = merged_df.sort_values(
    ['Model', 'scale_numeric', 'Benchmark', 'Technique']
).reset_index(drop=True)

print("✓ Data sorted")
merged_df.head(20)

✓ Data sorted


,Source paper,Technique,Model,Scale,Benchmark,Performance,scale_numeric
0,https://arxiv.org/pdf/2503.17365v1,Baseline,DeepSeek-R1 Distill Llama 8B,8B,HARMBENCH ASR,46.7,8.0
1,https://arxiv.org/pdf/2503.17365v1,ICL,DeepSeek-R1 Distill Llama 8B,8B,HARMBENCH ASR,8.9,8.0
2,https://arxiv.org/pdf/2503.17365v1,Baseline,Gemma-2 9B-it,9B,HARMBENCH ASR,58.9,9.0
3,https://arxiv.org/pdf/2503.17365v1,ICL,Gemma-2 9B-it,9B,HARMBENCH ASR,26.7,9.0
4,https://arxiv.org/pdf/2311.10702,Baseline,Llama 2-13B,13B,TOXIGEN,85.7,13.0
5,https://arxiv.org/pdf/2311.10702,SFT (V1 mix),Llama 2-13B,13B,TOXIGEN,18.7,13.0
6,https://arxiv.org/pdf/2311.10702,SFT (V2 mix),Llama 2-13B,13B,TOXIGEN,1.7,13.0
7,https://arxiv.org/pdf/2311.10702,Baseline,Llama 2-13B,13B,TRUTHFULQA,31.1,13.0
8,https://arxiv.org/pdf/2311.10702,SFT (V1 mix),Llama 2-13B,13B,TRUTHFULQA,45.3,13.0
9,https://arxiv.org/pdf/2311.10702,SFT (V2 mix),Llama 2-13B,13B,TRUTHFULQA,55.8,13.0


## Step 10: Summary Statistics

In [26]:
print("="*80)
print("MERGED DATASET SUMMARY")
print("="*80)

print(f"\nTotal data points: {len(merged_df)}")
print(f"\nUnique models: {merged_df['Model'].nunique()}")
print(f"  {', '.join(sorted(merged_df['Model'].unique())[:5])}...")

print(f"\nUnique benchmarks: {merged_df['Benchmark'].nunique()}")
print(f"  {', '.join(sorted(merged_df['Benchmark'].unique()))}")

print(f"\nUnique techniques: {merged_df['Technique'].nunique()}")
technique_counts = merged_df['Technique'].value_counts()
print("  Top 10 techniques by data point count:")
for tech, count in technique_counts.head(10).items():
    print(f"    - {tech}: {count} data points")

print(f"\nScale range: {merged_df['scale_numeric'].min():.0f}B - {merged_df['scale_numeric'].max():.0f}B")

print(f"\nBaseline data points: {len(merged_df[merged_df['Technique'] == 'Baseline'])}")

MERGED DATASET SUMMARY

Total data points: 163

Unique models: 17
  DeepSeek-R1 Distill Llama 8B, Gemma-2 9B-it, Llama 2-13B, Llama 2-70B, Llama 2-7B...

Unique benchmarks: 9
  DOANYTHINGNOW, HARMBENCH, HARMBENCH ASR, JAILBREAKTRIGGER, TOXIGEN, TRUTHFULQA, WILDGUARDTEST, WILDJAILBREAK, XSTEST

Unique techniques: 19
  Top 10 techniques by data point count:
    - Baseline: 42 data points
    - SFT: 22 data points
    - SFT + DPO: 19 data points
    - SFT + DPO + RLVR: 16 data points
    - SFT (V1 mix): 6 data points
    - SFT (QLORA): 6 data points
    - SFT (V2 mix): 6 data points
    - ICL: 4 data points
    - CPO: 4 data points
    - DPO: 4 data points

Scale range: 1B - 70B

Baseline data points: 42


## Step 10.5: Create Unique Technique Identifiers

**CRITICAL**: Techniques are NOT unique by name alone! 

The same technique name (e.g., "SFT") from different papers represents different implementations and should be treated as separate techniques.

We create a unique identifier by combining Technique + Source Paper hash.

## Step 11: Save Merged Dataset

In [27]:
output_path = '../data/merged_data.csv'
merged_df.to_csv(output_path, index=False)

print(f"✓ Saved to {output_path}")
print(f"\nFinal shape: {merged_df.shape}")
print(f"Columns: {list(merged_df.columns)}")

✓ Saved to ../data/merged_data.csv

Final shape: (163, 7)
Columns: ['Source paper', 'Technique', 'Model', 'Scale', 'Benchmark', 'Performance', 'scale_numeric']


## Verify Merged Data

In [28]:
# Quick verification
verification_df = pd.read_csv(output_path)

print("Verification of saved file:")
print(f"  Rows: {len(verification_df)}")
print(f"  Columns: {list(verification_df.columns)}")
print("\nFirst 10 rows:")
verification_df.head(10)

Verification of saved file:
  Rows: 163
  Columns: ['Source paper', 'Technique', 'Model', 'Scale', 'Benchmark', 'Performance', 'scale_numeric']

First 10 rows:


,Source paper,Technique,Model,Scale,Benchmark,Performance,scale_numeric
0,https://arxiv.org/pdf/2503.17365v1,Baseline,DeepSeek-R1 Distill Llama 8B,8B,HARMBENCH ASR,46.7,8.0
1,https://arxiv.org/pdf/2503.17365v1,ICL,DeepSeek-R1 Distill Llama 8B,8B,HARMBENCH ASR,8.9,8.0
2,https://arxiv.org/pdf/2503.17365v1,Baseline,Gemma-2 9B-it,9B,HARMBENCH ASR,58.9,9.0
3,https://arxiv.org/pdf/2503.17365v1,ICL,Gemma-2 9B-it,9B,HARMBENCH ASR,26.7,9.0
4,https://arxiv.org/pdf/2311.10702,Baseline,Llama 2-13B,13B,TOXIGEN,85.7,13.0
5,https://arxiv.org/pdf/2311.10702,SFT (V1 mix),Llama 2-13B,13B,TOXIGEN,18.7,13.0
6,https://arxiv.org/pdf/2311.10702,SFT (V2 mix),Llama 2-13B,13B,TOXIGEN,1.7,13.0
7,https://arxiv.org/pdf/2311.10702,Baseline,Llama 2-13B,13B,TRUTHFULQA,31.1,13.0
8,https://arxiv.org/pdf/2311.10702,SFT (V1 mix),Llama 2-13B,13B,TRUTHFULQA,45.3,13.0
9,https://arxiv.org/pdf/2311.10702,SFT (V2 mix),Llama 2-13B,13B,TRUTHFULQA,55.8,13.0
